In [1]:
import openmm
from forcebalance.openmmio import LocalEnergyMinimizer
import pathlib
from forcebalance.molecule import Molecule
from forcebalance.smirnoffio import OptGeoTarget_SMIRNOFF
from forcebalance.parser import parse_inputs
from forcebalance.forcefield import FF
from forcebalance.objective import Objective
from forcebalance.optimizer import Optimizer

In [2]:
TARGET_NAME = "19095588-8"

In [3]:
options, tgt_opts = parse_inputs("run-fb-target/optimize.in")

Reading options from file: run-fb-target/optimize.in
#========================================================#
#|   Options at their default values are not printed    |#
#|         Use 'verbose_options True' to Enable         |#
#========================================================#


In [4]:
forcefield = FF(options)

Reading force field from file: force-field.offxml
#=========================================================#
#|  Starting parameter indices, physical values and IDs  |#
#=========================================================#
   0 [  1.5339e+00 ] : Bonds/Bond/length/[#6X4:1]-[#6X4:2]
   1 [  4.3005e+02 ] : Bonds/Bond/k/[#6X4:1]-[#6X4:2]
   2 [  1.5094e+00 ] : Bonds/Bond/length/[#6X4:1]-[#6X3:2]
   3 [  4.7992e+02 ] : Bonds/Bond/k/[#6X4:1]-[#6X3:2]
   4 [  1.5296e+00 ] : Bonds/Bond/length/[#6X4:1]-[#6X3:2]=[#8X1+0]
   5 [  4.0435e+02 ] : Bonds/Bond/k/[#6X4:1]-[#6X3:2]=[#8X1+0]
   6 [  1.4677e+00 ] : Bonds/Bond/length/[#6X3:1]-[#6X3:2]
   7 [  5.3386e+02 ] : Bonds/Bond/k/[#6X3:1]-[#6X3:2]
   8 [  1.4008e+00 ] : Bonds/Bond/length/[#6X3:1]:[#6X3:2]
   9 [  7.5279e+02 ] : Bonds/Bond/k/[#6X3:1]:[#6X3:2]
  10 [  1.3731e+00 ] : Bonds/Bond/length/[#6X3:1]=[#6X3:2]
  11 [  9.0267e+02 ] : Bonds/Bond/k/[#6X3:1]=[#6X3:2]
  12 [  1.4757e+00 ] : Bonds/Bond/length/[#6:1]-[#7:2]
  13 [  4.5341e+02 

 540 [  1.0175e+01 ] : ImproperTorsions/Improper/k1/[*:1]~[#6X3:2](=[#7X2,#7X3+1:3])~[#7:4]
-----------------------------------------------------------
#=========================================================#
#|  Rescaling Factors by Type (Lower Takes Precedence):  |#
#=========================================================#
   ProperTorsions/Proper/k1             : 1.58682e+01
   ProperTorsions/Proper/k2             : 9.34193e+00
   ProperTorsions/Proper/k3             : 8.40574e+00
   ProperTorsions/Proper/k4             : 2.22152e+00
   ProperTorsions/Proper/k5             : 1.50805e+00
   ProperTorsions/Proper/k6             : 2.78965e+00
   ImproperTorsions/Improper/k1         : 1.68087e+01
   Angles/Angle/k                       : 1.00000e+02
   Angles/Angle/angle                   : 5.00000e+00
   Bonds/Bond/k                         : 1.00000e+02
   Bonds/Bond/length                    : 1.00000e-01
   ProperTorsions/Proper/k              : 5.00000e+00
   ImproperTorsions/

 540 [    ImproperTorsions/Improper/k   : 5.00000e+00 ] : ImproperTorsions/Improper/k1/[*:1]~[#6X3:2](=[#7X2,#7X3+1:3])~[#7:4]
----------------------------------------------------------
#========================================================#
#|                Setup for force field                 |#
#========================================================#
fnms                      ['force-field.offxml'] 
priors                    OrderedDict([('Angles/Angle/k', 100.0), ('Angles/Angle/angle', 5.0), ('Bonds/Bond/k', 100.0), ('Bonds/Bond/length', 0.1), ('ProperTorsions/Proper/k', 5.0), ('ImproperTorsions/Improper/k', 5.0)]) 
----------------------------------------------------------


In [5]:
def get_positions(options, tgt_opts, forcefield):
    from openmm import unit
    from copy import deepcopy
    import numpy as np
    from forcebalance.nifty import printcool_dictionary
    from forcebalance.openmmio import energy_components
    
    target = OptGeoTarget_SMIRNOFF(options, tgt_opts[0], forcefield)
    
    smirnoff_target = target.engines[TARGET_NAME]
    self = smirnoff_target
    
    smirnoff_target.update_simulation()
    
    with open('fb-193_system_reordered.xml') as input:
        self.system = openmm.XmlSerializer.deserialize(input.read())
    
    self.create_simulation(**self.simkwargs)
    smirnoff_target.set_positions(0)
    
    
    if self.restraint_frc_index is not None:
        self.set_restraint_positions(0)
    
    
    
    crit = 1e-4
    shot = 0
    steps = int(max(1, -1*np.log10(crit)))
    
    # Get the previous geometry.
    X0 = self.simulation.context.getState(getPositions=True).getPositions(asNumpy=True).value_in_unit(unit.angstrom)[self.realAtomIdxs]
    
    print(X0)
    printcool_dictionary(energy_components(self.simulation), title='Energy component analysis before minimization, shot %i' % shot)
    # Minimize the energy.  Optimizer works best in "steps".
    for logc in np.linspace(0, np.log10(crit), steps):
        e_minimized = self.simulation.context.getState(getEnergy=True).getPotentialEnergy().value_in_unit(unit.kilojoule_per_mole)
        print(e_minimized)
        self.simulation.minimizeEnergy(tolerance=10**logc * unit.kilojoule_per_mole, maxIterations=100000)
        
    # check if energy minimization is successful
    # try 1000 times with 10 steps each as openmm minimizer is not very stable at the tolerance
    for _ in range(1000):
        e_minimized = self.simulation.context.getState(getEnergy=True).getPotentialEnergy().value_in_unit(unit.kilojoule_per_mole)
        print(e_minimized)
        self.simulation.minimizeEnergy(tolerance=crit * unit.kilojoule_per_mole, maxIterations=10)
        e_new = self.simulation.context.getState(getEnergy=True).getPotentialEnergy().value_in_unit(unit.kilojoule_per_mole)
        if abs(e_new - e_minimized) < crit * 10:
            break
    else:
        logger.error("Energy minimization did not converge")
        raise RuntimeError("Energy minimization did not converge")
    # Remove the restraint energy from the total energy if desired.
    groups = set(range(32))
    if self.restraint_frc_index is not None and not include_restraint_energy:
        frc = self.simulation.system.getForce(self.restraint_frc_index)
        groups.remove(frc.getForceGroup())
    printcool_dictionary(energy_components(self.simulation), title='Energy component analysis after minimization, shot %i' % 0)
    S = self.simulation.context.getState(getPositions=True, getEnergy=True, groups=groups)
    # Get the optimized geometry.
    X1 = S.getPositions(asNumpy=True).value_in_unit(unit.angstrom)[self.realAtomIdxs]
    
    M = deepcopy(self.mol[0])
    M += deepcopy(M)
    M.xyzs = [X0, X1]
    if not self.pbc:
        M.align(center=False)
    X1 = M.xyzs[1]
    
    self._update_positions(X1, False)
    pos = self.getContextPosition()
    
    return pos

In [6]:
opt_positions = get_positions(options, tgt_opts, forcefield)

Reading optgeo options from file: targets/opt-geo-batch-113/optgeo_options.txt


system_mval_masks is built for faster gradient evaluations[[ 1.76616552  5.71298244  2.57919693]
 [ 1.2125943  -1.84077519  4.24200071]
 [ 3.11511612  6.02370788  2.75284706]
 [ 0.02936896 -1.25097856  3.77517986]
 [ 2.29635696  3.38919252  3.01982432]
 [ 0.25827974  0.51992146  5.43008713]
 [ 1.35499746  4.37933286  2.69692076]
 [ 1.91277314 -1.25757826  5.29480097]
 [ 1.42936338 -0.07754354  5.88262351]
 [ 4.07407399  5.04927843  3.0564807 ]
 [ 3.64473048  3.70614478  3.21614566]
 [-0.44729428 -0.0731784   4.37032014]
 [-1.88268468  2.89222824  2.34266568]
 [-2.20612809  4.21574103  2.42395319]
 [-0.03436427  3.97461757  2.52043333]
 [ 5.48378924 -2.79235752  5.75466484]
 [ 2.33643196  0.35724474  7.01429125]
 [ 3.62625574 -0.46335176  6.77598145]
 [ 3.17869548 -1.72507167  5.99233706]
 [ 5.52934223  5.43584335  3.16338396]
 [-3.47626274  4.98881231  2.42657984]
 [ 4.23643751  1.33227224  3.19007754]
 [ 5.09411548  2.77688388  4.92418657]
 [-2.77240603  1.68335806  2.23870766]
 [ 4.2

In [8]:
opt_positions

array([[ 2.19359055,  4.47167952,  4.29521979],
       [ 0.85074453, -1.4524838 ,  4.61108287],
       [ 3.55797548,  4.72177116,  4.53394407],
       [-0.40445842, -1.36947253,  3.97042495],
       [ 2.79761299,  2.64966637,  2.78306388],
       [-0.8985186 ,  0.688681  ,  5.19299223],
       [ 1.80682506,  3.44771856,  3.40323522],
       [ 1.24974011, -0.42283545,  5.49251075],
       [ 0.37244207,  0.62238251,  5.79617192],
       [ 4.55417168,  3.95373811,  3.8880867 ],
       [ 4.17515237,  2.87972371,  3.02317992],
       [-1.26060259, -0.27354269,  4.22885597],
       [-1.47685   ,  2.4272253 ,  2.40144663],
       [-1.77967941,  3.40642729,  3.31330083],
       [ 0.3670564 ,  3.21141382,  3.13482817],
       [ 5.13890703, -0.50048419,  5.98474912],
       [ 0.98552106,  1.58312429,  6.773542  ],
       [ 2.48910346,  1.21681489,  6.72281399],
       [ 2.56301463, -0.26041527,  6.22372433],
       [ 6.00730649,  4.36085055,  4.10155172],
       [-3.13347488,  3.86707566,  3.815